In [1]:
import pandas as pd
import sys
import os
sys.path.append("/Users/marc/Desktop/PostDoc/Mutation/signature5/notebooks")
import tools

col_order = ['Chr', 'Pos', 'Ref', 'Alt', 'Sample', 'Tissue_CellType', 'Compartment',
             'Individual', 'Study', 'Context', 'GeneralType', 'Type']
col_meta = ["Study", "Sample", "Individual", "Age", "Compartment", "Status", "Tissue_CellType"]

Read data

In [2]:
dat = pd.read_csv("Abascal2021.pre.csv.gz")

In [25]:
dat["Sample"] = dat.Cell
dat["Individual"] = dat.Cell.str[:7]

In [ ]:
cosmic = tools.COSMIC()
dat = tools.annotate_context(dat, "../fasta/hg19.fa")
dat = dat.assign(GeneralType=tools.annotate_general_type(dat))
dat['Type'] = cosmic.fold(dat['GeneralType'])

In [30]:
pmeta = pd.read_excel("41586_2021_3477_MOESM3_ESM.xlsx",sheet_name="SupTable1 - Samples")

In [38]:
dat["Tissue_CellType"] = dat.Cell.str.split("_").str[1].str.capitalize()

In [41]:
dat["Compartment"] = "WGS"

In [42]:
dat[col_order].to_csv("Abascal2022.csv.gz", compression="gzip")

Meta file

In [49]:
pmeta["Status"] = pmeta["Disease state"].replace("-", "Normal")

In [56]:
ages = pmeta.set_index("Donor id")["Age"].to_dict()
stat = pmeta.set_index("Donor id")["Status"].to_dict()
tissue = dat.set_index("Sample")["Tissue_CellType"].to_dict()
cell2indv = dat.set_index("Sample")["Individual"].to_dict()

In [57]:
l = list(dat.Sample.unique())
meta = pd.DataFrame(l)
meta.columns = ["Sample"]
meta["Study"] = "Abascal2021"
meta["Tissue_CellType"] = meta.Sample.map(tissue)
meta["Individual"] = meta.Sample.map(cell2indv)
meta["Age"] = meta["Individual"].map(ages)
meta["Status"] = meta["Individual"].map(stat)
meta["Compartment"] = "WGS"

In [60]:
meta[col_meta].to_csv("Abascal2021.tab.gz",sep="\t", index=False)